In [85]:
import pandas as pd
import numpy as np

import os
import sys
import json
import toml

import snowflake
from snowflake.snowpark import Session
from snowflake.core import Root
from snowflake.cortex import Complete

from trulens.apps.custom import instrument, TruCustomApp
from trulens.core.instruments import WithInstrumentCallbacks

In [2]:
df = pd.read_csv('wine_reviews.csv')

In [4]:
df.description.apply(lambda x: len(x)).describe()


count    81115.000000
mean       247.411502
std         68.857579
min         53.000000
25%        200.000000
50%        239.000000
75%        284.000000
max        754.000000
Name: description, dtype: float64

In [10]:

try:
    session = get_active_session()
    print("using get_active_session")
except:
    config = toml.load('/Users/ebotwick/library/Application Support/snowflake/config.toml').get('connections').get('ebotwick_default')
    session = Session.builder.configs(config).create()
    print("connected to snowflake!")

In [13]:
session.use_database("WINE_REVIEW_LLM")
session.use_schema("DATA")

In [24]:
session.write_pandas(df, 
                     table_name="WINE_REVIEWS_RAW",
                     auto_create_table=True, 
                     quote_identifiers=False,
                     overwrite=True)

In [25]:
dfsp = session.table("WINE_REVIEWS_RAW")
print(list(dfsp.schema))

[StructField('COUNTRY', StringType(16777216), nullable=True), StructField('DESCRIPTION', StringType(16777216), nullable=True), StructField('DESIGNATION', StringType(16777216), nullable=True), StructField('POINTS', LongType(), nullable=True), StructField('PRICE', DoubleType(), nullable=True), StructField('PROVINCE', StringType(16777216), nullable=True), StructField('REGION_1', StringType(16777216), nullable=True), StructField('REGION_2', StringType(16777216), nullable=True), StructField('TASTER_NAME', StringType(16777216), nullable=True), StructField('TASTER_PHOTO', StringType(16777216), nullable=True), StructField('TASTER_TWITTER_HANDLE', StringType(16777216), nullable=True), StructField('TITLE', StringType(16777216), nullable=True), StructField('VARIETY', StringType(16777216), nullable=True), StructField('VINTAGE', StringType(16777216), nullable=True), StructField('WINERY', StringType(16777216), nullable=True)]


In [35]:
dfsp.show(10)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"COUNTRY"  |"DESCRIPTION"                                       |"DESIGNATION"           |"POINTS"  |"PRICE"  |"PROVINCE"                |"REGION_1"   |"REGION_2"     |"TASTER_NAME"      |"TASTER_PHOTO"                                      |"TASTER_TWITTER_HANDLE"  |"TITLE"                                             |"VARIETY"           |"VINTAGE"  |"WINERY"           |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
session.sql("SHOW TABLES").collect()

[Row(created_on=datetime.datetime(2024, 12, 28, 19, 30, 47, 343000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), name='WINE_REVIEWS_RAW', database_name='WINE_REVIEW_LLM', schema_name='DATA', kind='TABLE', comment='', cluster_by='', rows=81115, bytes=9870848, owner='ACCOUNTADMIN', retention_time='1', automatic_clustering='OFF', change_tracking='OFF', search_optimization='OFF', search_optimization_progress=None, search_optimization_bytes=None, is_external='N', enable_schema_evolution='N', owner_role_type='ROLE', is_event='N', budget=None, is_hybrid='N', is_iceberg='N', is_dynamic='N', is_immutable='N')]

In [41]:
create_css_sql = '''
CREATE OR REPLACE CORTEX SEARCH SERVICE WINE_REVIEW_RETRIEVER 
ON DESCRIPTION
ATTRIBUTES PRICE, POINTS, COUNTRY, VARIETY, VINTAGE
warehouse = DEFAULT_XS
TARGET_LAG = '1 day'
as (
    select DESCRIPTION, 
    PRICE, 
    POINTS, 
    COUNTRY, 
    VARIETY, 
    VINTAGE
    from WINE_REVIEWS_RAW
);

'''

session.sql(create_css_sql).collect()

[Row(status='Cortex search service WINE_REVIEW_RETRIEVER successfully created.')]

In [51]:
test_css_sql = '''SELECT PARSE_JSON(SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'WINE_REVIEW_RETRIEVER', 
    '{
      "query": "whats a good merlot from california"
    }'
))'''



session.sql(test_css_sql).collect()

[Row(PARSE_JSON(SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
     'WINE_REVIEW_RETRIEVER', 
     '{
       "QUERY": "WHATS A GOOD MERLOT FROM CALIFORNIA"
     }'
 ))='{\n  "request_id": "0706fa97-fca7-4fd3-a610-4f25cc6cbf4b",\n  "results": [\n    {\n      "@DEBUG_PER_RESULT": "{\\"TopicalityScore\\":0.014335629,\\"L2DScore\\":1.3960292,\\"Q2DScore\\":-2.379537,\\"Q2DScoreNorm\\":1,\\"L2DScoreNorm\\":0.9362315,\\"FinalScore\\":0.59762114}",\n      "DESCRIPTION": "Though not treating the grape with quite the same reverence as other varieties, Central Coast vintners are having fun with Merlot, making quaffable, easygoing bottlings like this one. It shows caramelized blueberry, black currant and asphalt touches on the nose. The graphite minerality also pops on the palate, giving a tension to the cassis flavors. "\n    },\n    {\n      "@DEBUG_PER_RESULT": "{\\"TopicalityScore\\":0.023584815,\\"L2DScore\\":1.3507818,\\"Q2DScore\\":-4.3525505,\\"Q2DScoreNorm\\":0.70625263,\\"L2DScoreNorm\\":0.5652181,\\

In [67]:
root = Root(session)                         
svc = root.databases[session.get_current_database()].schemas[session.get_current_schema()].cortex_search_services["WINE_REVIEW_RETRIEVER"]

In [68]:
SEARCH_COLS = ["DESCRIPTION"]
svc.search(query='whats a good california white', columns = SEARCH_COLS, limit = 5)

QueryResponse(results=[{'DESCRIPTION': 'This white has a concentrated richness of Meyer lemon and melon, the palate rounded and lush. From an estate site at some elevation, it is a rarity to find this variety in California; this is a good place to start. ', '@DEBUG_PER_RESULT': '{"TopicalityScore":0.08852696,"L2DScore":1.24225,"Q2DScore":-2.4208546,"Q2DScoreNorm":0.9570702,"L2DScoreNorm":1,"TopicalityScoreNorm":1,"FinalScore":0.60051966}'}, {'DESCRIPTION': 'This bottling is evidence of the strides winemaker David Baird has made in recent years to uplift this new brand. This white is what classic California Chardonnays should taste like when made in a balanced way. Aromas of butter, lemon peels, gardenia and chalk lead into toast, mango, pineapple and peach on the palate that hide behind an earthy, crushed, white-rock structure. ', '@DEBUG_PER_RESULT': '{"TopicalityScore":0.038456667,"L2DScore":1.2269329,"Q2DScore":-3.0867498,"Q2DScoreNorm":0.8712836,"L2DScoreNorm":0.8899371,"Topicality

In [79]:
class WINE_RAG_APP():

    def __init__(self, llm_model, search_service, num_chunks):
        self.llm_model = llm_model
        self.search_service = search_service
        self.num_chunks = num_chunks

    @instrument
    def retrieve(self, myquestion: str):

        # if WEEK_NUM == "ALL":
        response = self.search_service.search(myquestion, SEARCH_COLS, limit=self.num_chunks)
        # else: 
            # filter_obj = {"@eq": {"WEEK_NUM": str(WEEK_NUM)} }
            # response = self.search_service.search(myquestion, SEARCH_COLS, filter=filter_obj, limit=num_chunks)
    
        # st.sidebar.json(response.json())
        # st.markdown(json.loads(response.json()).get('results'))
        return [i.get("DESCRIPTION") for i in json.loads(response.json()).get('results')]
        # return ["HOME_TEAM: " +i.get("HOME_TEAM")+ "\nAWAY_TEAM: "+i.get("AWAY_TEAM")+"\n" +i.get("CHUNKED_TEXT") for i in json.loads(response.json()).get('results')]

    @instrument
    def create_prompt (self, myquestion: str):
        if rag == 1:
            prompt_context = self.retrieve(myquestion)
      
            prompt = f"""
               You are an expert Wine chat assistant. I'll call you SomoLLMier
               Your job is to extract information from the CONTEXT provided
               between <context> and </context> tags.
               When answering the question contained between <question> and </question> tags
               be concise and do not hallucinate. 
               If you don´t have the information just say so.
               Only anwer the question if you can extract it from the CONTEXT provideed.
               
        
               <context>          
               {prompt_context}
               </context>
               <question>  
               {myquestion}
               </question>
               Answer: 
               """            
        else:     
            prompt = f"""[0]
             'Question:  
               {myquestion} 
               Answer: '
               """
            # relative_paths = "None"
            prompt_context = "None"
                
        return prompt, prompt_context
    @instrument
    def complete_prompt(self, myquestion: str, stream: bool):
    
        prompt, retrieved_context =self.create_prompt(myquestion)
        
        df_response = Complete(self.llm_model, prompt, stream=stream)
        return df_response

In [83]:
rag=1
wine_rag = WINE_RAG_APP('mistral-7b', svc, 5)


In [88]:
wine_rag.complete_prompt("when someone calls a wine delicate what does that mean", stream=False)

' In the context provided, a delicate wine is described as having light body, subtle aromas, and a refined character. It is often easy to drink and may have a gentle, understated flavor profile.'